In [47]:
import sys
import os

# Add the path src/preprocessing.py
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [49]:
import pandas as pd
data = pd.read_csv("../data/processed/eda_output.csv")
print(data.head())

  label                                            message  message_length
0   ham  Go until jurong point, crazy.. Available only ...             111
1   ham                      Ok lar... Joking wif u oni...              29
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...             155
3   ham  U dun say so early hor... U c already then say...              49
4   ham  Nah I don't think he goes to usf, he lives aro...              61


In [55]:
from src.preprocessing import preprocess_text

# Create a new column in the data with messages preprocessing 
data['cleaned_message'] = data['message'].apply(preprocess_text)

print(data[['message', 'cleaned_message']].head(5))



                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                     cleaned_message  
0  go jurong point crazy available bugis n great ...  
1                            ok lar joking wif u oni  
2  free entry wkly comp win fa cup final tkts st ...  
3                u dun say early hor u c already say  
4           nah dont think go usf life around though  
